In [1]:
pip install pefile yara-python

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 9.1 MB/s eta 0:00:00

   -------------------- ------------------- 1/2 [pefile]
   ---------------------------------------- 2/2 [pefile]

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Static triage full script for Procmon.exe
# Save this file and paste into a Jupyter Notebook cell (or run as a .py script).
# Required packages:
# pip install pefile yara-python

import hashlib
import pefile
import re
import yara
import os
import sys

# --- Configuration ---------------------------------------------------------
# Replace this path with the full path to your Procmon.exe sample
sample = r"C:\Users\awans\Downloads\ProcessMonitor\Procmon.exe"

# Minimum printable ASCII string length to extract
MIN_STR_LEN = 4

# ---------------------------------------------------------------------------

def compute_hashes(path):
    """Compute MD5, SHA1 and SHA256 for the file at `path`.
    Returns a dict {algo: hexdigest}.
    """
    algos = ["md5", "sha1", "sha256"]
    output = {}
    for a in algos:
        h = hashlib.new(a)
        with open(path, "rb") as f:
            # read in chunks to avoid huge memory use for very large files
            for chunk in iter(lambda: f.read(8192), b""):
                h.update(chunk)
        output[a] = h.hexdigest()
    return output


def extract_strings(path, min_len=MIN_STR_LEN):
    """Extract printable ASCII strings of length >= min_len from the binary file.
    Returns a list of byte-strings.
    """
    pattern = rb"[ -~]{%d,}" % (min_len,)
    with open(path, "rb") as f:
        data = f.read()
    return re.findall(pattern, data)


def parse_pe_imports(path):
    """Parse PE headers and return a summary dict with entry point, image base,
    and a list of imported DLLs and a sample of imported functions.
    """
    pe_info = {
        "entry_point": None,
        "image_base": None,
        "imports": []
    }
    try:
        pe = pefile.PE(path)
    except Exception as e:
        pe_info["error"] = f"PE parsing failed: {e}"
        return pe_info

    try:
        ep = pe.OPTIONAL_HEADER.AddressOfEntryPoint
        ib = pe.OPTIONAL_HEADER.ImageBase
        pe_info["entry_point"] = hex(ep)
        pe_info["image_base"] = hex(ib)
    except Exception:
        # optional header may be missing/unexpected
        pass

    try:
        for entry in getattr(pe, "DIRECTORY_ENTRY_IMPORT", []) or []:
            dll_name = entry.dll.decode(errors="ignore") if entry.dll else ""
            funcs = []
            for imp in entry.imports[:10]:
                try:
                    funcs.append(imp.name.decode(errors="ignore") if imp.name else "<ordinal>")
                except Exception:
                    funcs.append("<unk>")
            pe_info["imports"].append({"dll": dll_name, "sample_funcs": funcs})
    except Exception:
        # reading imports may fail for corrupted/packed binaries
        pass

    return pe_info


def find_iocs_from_strings(strings):
    """Given a list of decoded strings (python str), find URLs and IP addresses.
    Returns dict with lists for 'urls' and 'ips'.
    """
    joined = "\n".join(strings)
    urls = re.findall(r"https?://[^\s\"']+", joined)
    ips = re.findall(r"\b\d{1,3}(?:\.\d{1,3}){3}\b", joined)
    return {"urls": urls, "ips": ips}


def yara_check(path, source=None):
    """Compile a YARA rule from source (string) or use a default simple rule
    that looks for the literal "http". Return list of matches (could be empty).
    """
    if source is None:
        source = '''
rule SimpleHTTP {
    strings:
        $s = "http"
    condition:
        $s
}
'''
    try:
        rules = yara.compile(source=source)
        matches = rules.match(path)
        return matches
    except yara.SyntaxError as e:
        return f"YARA syntax error: {e}"
    except Exception as e:
        return f"YARA error: {e}"


# ------------------------- Main workflow -----------------------------------

def run_full_triage(path):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Sample file not found: {path}")

    print("\n=== STATIC TRIAGE: file:", path)

    # 1) Hashes
    print("\n-- HASHES --")
    hashes = compute_hashes(path)
    for k, v in hashes.items():
        print(f"{k.upper():6}: {v}")

    # 2) Strings
    print("\n-- STRINGS (first 50) --")
    raw_strings = extract_strings(path)
    # decode safely, ignoring errors
    decoded = [s.decode(errors="ignore") for s in raw_strings]
    for s in decoded[:50]:
        print(s)

    # 3) PE Header / imports
    print("\n-- PE HEADER & IMPORTS --")
    pe_info = parse_pe_imports(path)
    if "error" in pe_info:
        print(pe_info["error"])
    else:
        print("Entry Point:", pe_info.get("entry_point"))
        print("Image Base:", pe_info.get("image_base"))
        print("Imported DLLs (sample):")
        for imp in pe_info.get("imports", [])[:20]:
            print(" ", imp["dll"], "->", imp.get("sample_funcs", [])[:5])

    # 4) IOCs from strings
    print("\n-- IOCs from strings --")
    iocs = find_iocs_from_strings(decoded)
    print("URLs:", iocs.get("urls"))
    print("IPs:", iocs.get("ips"))

    # 5) YARA
    print("\n-- YARA RULES --")
    yara_result = yara_check(path)
    print("YARA matches:", yara_result)

    print("\n=== END OF TRIAGE ===\n")


if __name__ == "__main__":
    target = sample
    if len(sys.argv) > 1:
        target = sys.argv[1]
    try:
        run_full_triage(target)
    except Exception as e:
        print(f"Error during triage: {e}")


# Notes for Jupyter Notebook usage:
# - Put the entire script into a single notebook cell and run it.



Error during triage: Sample file not found: -f


In [ ]:
run_full_triage(sample)
